In [1]:
# conda install -c conda-forge bayesian-optimization
# conda install -c conda-forge xgboost

In [2]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
# from xgboost import XGBClassifier

In [3]:
# Reading the saved dtypes Series
final_df_dtypes = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_fromBDSE12_03G_HomeCredit_V2_20200204a_dtypes_series.csv'\
            , header=None, index_col=0, squeeze=True)
del final_df_dtypes.index.name
final_df_dtypes = final_df_dtypes.to_dict()

final_df = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_fromBDSE12_03G_HomeCredit_V2_20200204a.csv'\
           , dtype= final_df_dtypes)

In [4]:
final_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in final_df.columns]
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 4081 entries, SK_ID_CURR to GOODS_PRICE_PREV__na
dtypes: float64(543), int64(4), uint8(3534)
memory usage: 2.6 GB


In [5]:
df = final_df
del final_df
gc.collect()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 4081 entries, SK_ID_CURR to GOODS_PRICE_PREV__na
dtypes: float64(543), int64(4), uint8(3534)
memory usage: 2.6 GB


In [6]:
df.index.size

356255

---

## LGBM

In [7]:
gc.collect()

44

In [8]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
    params['min_data_in_leaf'] = int(params['min_data_in_leaf'])
    params['max_bin'] = int(params['max_bin'])
    
        
    clf = LGBMClassifier(**params, 
                         n_estimators = 2000,
                         nthread = 4, 
                         boosting_type='goss', 
                         objective='binary')

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [9]:
init_time = time.time()
params = {'learning_rate': (.01, .1), 
          'num_leaves': (21, 99), 
          'subsample': (0.7, 1), 
          'max_depth': (6, 21), 
          'reg_alpha': (.00, 1.0), 
          'reg_lambda': (.00, 1.0), 
          'min_split_gain': (.01, .1),
          'min_child_weight': (20, 70),
          'min_data_in_leaf': (20, 70),
          'max_bin': (55, 255)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 5, n_iter = 5)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

|   iter    |  target   | learni... |  max_bin  | max_depth | min_ch... | min_da... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7877   |  0.09139  |  214.0    |  16.39    |  41.66    |  54.79    |  0.0679   |  56.43    |  0.9324   |  0.9778   |  0.765    |
|  2        |  0.7886   |  0.05715  |  72.75    |  16.8     |  55.46    |  60.48    |  0.0931   |  96.92    |  0.573    |  0.5866   |  0.7089   |
|  3        |  0.7931   |  0.01847  |  116.5    |  15.17    |  62.45    |  26.67    |  0.0713   |  33.71    |  0.5439   |  0.8294   |  0.9961   |
|  4        |  0.7891   |  0.08362  |  210.5    |  10.85    |  69.9     |  44.43    |  0.09261  |  54.55    |  0.5237   |  0.2922   |  0.7317   |
|  5        |  0.7934   |  0.02304  |  199.1    |  10.21    |  49.45    |  61.69    |  0.02764  |  24.05    |  0.5398   |  0

In [12]:
bo.res[4]

{'target': 0.7933952317104458,
 'params': {'learning_rate': 0.023043339824901234,
  'max_bin': 199.11806433024913,
  'max_depth': 10.210096015024956,
  'min_child_weight': 49.45247211242131,
  'min_data_in_leaf': 61.69014344600048,
  'min_split_gain': 0.027635633365652647,
  'num_leaves': 24.050268842962076,
  'reg_alpha': 0.5398457465148843,
  'reg_lambda': 0.018724331349439294,
  'subsample': 0.9829092682943514}}

In [13]:
type(bo.res[4])

dict

In [14]:
params_list = [bo.res[i] for i in range(10)]

[{'target': 0.7876940542151684,
  'params': {'learning_rate': 0.09138562910257374,
   'max_bin': 213.95626320247877,
   'max_depth': 16.388746706192755,
   'min_child_weight': 41.66314075886564,
   'min_data_in_leaf': 54.79200045417029,
   'min_split_gain': 0.06789912220539994,
   'num_leaves': 56.43162958207656,
   'reg_alpha': 0.932392918588331,
   'reg_lambda': 0.9778012435351334,
   'subsample': 0.7650304453850436}},
 {'target': 0.7886484305970665,
  'params': {'learning_rate': 0.05715084141400458,
   'max_bin': 72.74514674492127,
   'max_depth': 16.804595811071156,
   'min_child_weight': 55.462697433761825,
   'min_data_in_leaf': 60.4848622054932,
   'min_split_gain': 0.09310379562594918,
   'num_leaves': 96.92385260396496,
   'reg_alpha': 0.5730189815430475,
   'reg_lambda': 0.5866366407740554,
   'subsample': 0.7088956888561413}},
 {'target': 0.7931267365648386,
  'params': {'learning_rate': 0.01846829743386448,
   'max_bin': 116.54868247698147,
   'max_depth': 15.17275284263175

In [3]:
# 輸出參數檔案
import json
with open('params_list.txt', 'w', encoding='utf-8') as fout:
    for params in params_list:
        json.dump(params, fout) 
        fout.write("\n")

In [6]:
# 讀取參數檔案
with open('params_list.txt', 'r', encoding='utf-8') as f:
    params_list_read = list(map(json.loads,f))

In [ ]:
init_time = time.time()
params = {'learning_rate': (.0, .1), 
          'num_leaves': (20, 100), 
          'subsample': (.0, 1.0), 
          'max_depth': (6, 9), 
          'reg_alpha': (.00, 1.0), 
          'reg_lambda': (.00, 1.0), 
          'min_split_gain': (.0, .1),
          'min_child_weight': (20, 70)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 5, n_iter = 10)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

---

## XGboost

In [ ]:
def xgb_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['max_depth'] = int(params['max_depth'])
        
    clf = XGBClassifier(**params, 
                        n_estimators = 2000, 
                        nthread = 5, 
                        objective= 'binary:logistic')

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [ ]:
init_time = time.time()
params = {'learning_rate': (.01, .03), 
          'subsample': (.0, 1.0), 
          'max_depth': (4, 9), 
          'reg_alpha': (.0, 1.0), 
          'reg_lambda': (.0, 1.0), 
          'scale_pos_weight': (.0, 5.0),
          'colsample_bytree': (.0, 1.0)}
bo = BayesianOptimization(xgb_evaluate, params)
bo.maximize(init_points = 5, n_iter = 5)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))